In [1]:
import numpy as np
from astropy.table import Table, vstack
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
from tqdm import tqdm


import os
import psycopg2

from astropy import units as u
from astropy.table import Table, join, unique, vstack
from astropy.coordinates import SkyCoord, match_coordinates_sky, search_around_sky

from desitarget.targets import decode_targetid, encode_targetid, resolve
from desitarget.io import releasedict, release_to_photsys

from tqdm.notebook import tqdm_notebook

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt


from astropy.table import Table, vstack, hstack

import psycopg2

from tqdm.notebook import tqdm_notebook

import time

In [4]:
t = Table.read('/pscratch/sd/n/nravi/pv_rot_curves/iron_not_jura.fits')
t[:5]

TARGETID,SURVEY,FILENAME
float64,bytes4,bytes65
1079550230396930.0,sv3,iron/healpix/sv3/bright/103/10378/redrock-sv3-bright-10378.fits
1092744369930242.0,sv3,iron/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits
1079550234591232.0,sv3,iron/healpix/sv3/bright/103/10378/redrock-sv3-bright-10378.fits
1092744374124544.0,sv3,iron/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits
1092755086376960.0,sv3,iron/healpix/sv3/dark/103/10379/redrock-sv3-dark-10379.fits


In [2]:
def get_tf_targets_modified(redux, targetid=None, use_cached=False, verbose=False):
    """Get TF targets from the DESI observations DB for a given spectroscopic reduction.
    
    Parameters
    ----------
    redux : str
        Spectroscopic reduction. E.g., 'everest', 'fuji', ...
    use_cached : bool
        Use cached data rather than re-running the query.
    
    Returns
    -------
    
    tf_targets : Table
        Table of Tully-Fisher observations.
    """
    tf_targets = None

    if os.path.exists(f'tf_targets_{redux}.fits') and use_cached:
        tf_targets = Table.read('cache/tf_targets_{redux}.fits')
    else:
        try:
            db = psycopg2.connect(host='decatdb.lbl.gov', database='desidb', user='desi', password='5kFibers!')
            cursor = db.cursor()

            query = f"""SELECT rdx.targetid, rdx.target_ra, rdx.target_dec, rdx.mean_fiber_ra, rdx.mean_fiber_dec, rdx.std_fiber_ra, rdx.std_fiber_dec, zd.z, zd.zerr, zd.spectype, zd.deltachi2, zd.zwarn, pv.pvtype, pv.sga_id
                   FROM {redux}.healpix_fibermap as rdx, static.pv as pv, {redux}.healpix_redshifts as zd
                   WHERE q3c_join(rdx.target_ra, rdx.target_dec, pv.ra, pv.dec, 1./3600.) 
                         AND zd.targetid = rdx.targetid;"""
                         # AND pv.sga_id IS NOT NULL AND (pv.pvtype LIKE 'TFT' or pv.pvtype LIKE 'EXT' or pv.pvtype LIKE 'SGA');"""
            
            if verbose:
                print(query)

            cursor.execute(query)
            rows = cursor.fetchall()
            tf_targets = Table(list(map(list, zip(*rows))),
                               names=['TARGETID', 'TARGET_RA', 'TARGET_DEC', 'MEAN_FIBER_RA', 'MEAN_FIBER_DEC','STD_FIBER_RA', 'STD_FIBER_DEC', 'Z', 'ZERR', 'SPECTYPE', 'DELTACHI2', 'ZWARN', 'PVTYPE', 'SGA_ID'])

            #- Select only targets with SGA IDs and PV types matching SGA, EXT, and TFT
            print('selecting')
            select = (tf_targets['SGA_ID'] != None) & (tf_targets['TARGETID'] == targetid) &\
                     ((tf_targets['PVTYPE'] == 'TFT') | \
                      (tf_targets['PVTYPE'] == 'EXT') | \
                      (tf_targets['PVTYPE'] == 'SGA') )
            tf_targets = tf_targets[select]
            print('done')
            #- Use TARGETID to extract the photometric system used during targeting
            _, _, releases, _, _, _ = decode_targetid(tf_targets['TARGETID'].value)

            photsys = []
            for i, release in enumerate(releases):
                ps = None

                if release in releasedict:
                    ps = release_to_photsys([release])[0].decode('utf-8')
                else:
                    #- Fall-through case: not all SGA center observations are in the main survey.
                    #  In this case, select 'N' or 'S' based on the SGA object's position.
                    ra  = tf_targets['TARGET_RA'][i]
                    dec = tf_targets['TARGET_DEC'][i]
                    c = SkyCoord(ra=ra, dec=dec, unit='degree')

                    #- N: in galactic northern hemisphere and with dec > 32.375. Else, S.
                    isnorth = (c.galactic.b > 0) & (dec > 32.375)
                    ps = 'N' if isnorth else 'S'

                photsys.append(ps)

            #- Complain if the photsys table doesn't match the size of the Vrot table.
            if len(photsys) != len(tf_targets):
                print(f'photsys array of len {len(photsys)} != targets array of len {len(tf_targets)}')

            tf_targets['PHOTSYS'] = photsys

            # tf_targets.write(f'cache/tf_targets_{redux}.fits', overwrite=True)

        except Exception as error:
            print(error)
        finally:
            if db is not None:
                db.close()

    return tf_targets

In [ ]:
match = get_tf_targets_modified('jura', verbose=True)

SELECT rdx.targetid, rdx.target_ra, rdx.target_dec, rdx.mean_fiber_ra, rdx.mean_fiber_dec, rdx.std_fiber_ra, rdx.std_fiber_dec, zd.z, zd.zerr, zd.spectype, zd.deltachi2, zd.zwarn, pv.pvtype, pv.sga_id
                   FROM jura.healpix_fibermap as rdx, static.pv as pv, jura.healpix_redshifts as zd
                   WHERE q3c_join(rdx.target_ra, rdx.target_dec, pv.ra, pv.dec, 1./3600.) 
                         AND zd.targetid = rdx.targetid;
